In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import IPython

from keras.models import Sequential
from keras.layers import Dense, Activation, Conv2D, SeparableConv2D, MaxPooling2D, AveragePooling2D, GlobalMaxPooling2D, BatchNormalization, Flatten, Dropout, InputLayer
from keras.optimizers import Adam, Adamax, RMSprop
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator

## Import data from Google Drive 

In [0]:
# __import__('google.colab.files', fromlist=[None]).upload() # too slow to upload
# __import__('google.colab.drive', fromlist=[None]).mount('/content/gdrive') # mount the entire drive

!pip install PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

download = drive.CreateFile({'id': '1D7NEp6r4LslueEGRWrTDVE7s_61i2RW3'})
download.GetContentFile('data_img.zip')

!unzip data_img.zip

del auth, gauth, drive, download

IPython.display.clear_output()

total 309M
drwxr-xr-x 1 root root 4.0K Jan 28 21:40 .
drwxr-xr-x 1 root root 4.0K Jan 28 21:39 ..
-rw-r--r-- 1 root root 2.5K Jan 28 21:40 adc.json
drwxr-xr-x 1 root root 4.0K Jan 28 21:40 .config
-rw-r--r-- 1 root root 309M Jan 28 21:40 data_img.zip
drwxr-xr-x 1 root root 4.0K Jan  8 17:15 sample_data
